In [1]:
import sys
sys.path.insert(0, '../')
from qhdopt import QHD
import numpy as np
import mpmath

/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [14]:
Q = np.array([[0.8931, 0.8673],
     [0.8673, 0.9079]])
b = np.array([0.8483, 0.8762])

In [2]:
def to_numpy(v):
    return np.array(v.tolist(), dtype=np.float64)

In [37]:
def run_qhd(matrix, vector):
    # first_nonzero_decimal_place = min([get_first_nonzero_decimal_place(v_i) for v_i in vector])
    # if first_nonzero_decimal_place <= 1:
    #     offset = 0
    # else:
    #     offset = first_nonzero_decimal_place - 1
    vector = to_numpy(-1 * vector).squeeze()
    model = QHD.QP(matrix, vector, bounds=(-1, 1))
    model.dwave_setup(20, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt', embedding_scheme="hamming")
    model.optimize(verbose=0, fine_tune=False)
    return model.info["coarse_minimizer_affined"]


def IR_QHD(matrix, vector, IRprecision):
  ### Scaled Iterative Refinement for solving a linear system
  ### matrix: coefficient matrix of the linear system
  ### vector: right-hand side vector of the linear system
    nabla             = 1                             # Scaling factor
    rho               = 4                             # Incremental scaling
    d                 = len(matrix)                   # Dimension
    iteration         = 0
    x                 = mpmath.matrix(np.zeros(d))                 # Solution
    r                 = vector                        # Residual
    # con               = np.linalg.cond(matrix)             # Condition number
    res=[]
    while (np.linalg.norm(r)>IRprecision):
        c = run_qhd(to_numpy(matrix), nabla*r)
        c = mpmath.matrix(c)
        x = x + (1/nabla)*c                           # Updating solution
        r = vector - matrix * x               # Calculating resisdual
        nabla = min(rho*nabla,1/(np.linalg.norm(r)))  # Updating scaling factor
        res.append(np.linalg.norm(r))
        iteration=iteration+1
    return res, c

# Set the desired precision (number of decimal places)
mpmath.mp.dps = 50  # for example, 50 decimal places

# Create the matrix A using numpy
# A = np.random.rand(2, 2)
A = np.array([[2, 0], [0, 3]])
# Convert A to mpmath matrix and make it symmetric to ensure positive-definiteness
A_mpmath = mpmath.matrix(A)
# A_mpmath = A_mpmath * A_mpmath.transpose()

# High precision x vector
x_mpmath = mpmath.matrix([0.5, 0.33333333333333333333])

# Calculate b using mpmath's dot product
b_mpmath = A_mpmath * x_mpmath

# Convert results back to numpy arrays for any further processing
A_np = np.array(A_mpmath.tolist(), dtype=np.float64)
b_np = np.array(b_mpmath.tolist(), dtype=np.float64)


print(IR_QHD(A_mpmath, b_mpmath, 1e-16))

Received Task from D-Wave:
2024-02-05 03:13:15
Backend QPU Time: 0.02581037
Overhead Time: 3.39160644098938

Received Task from D-Wave:
2024-02-05 03:13:19
Backend QPU Time: 0.027283169999999995
Overhead Time: 3.6209187724438476

Received Task from D-Wave:
2024-02-05 03:13:23
Backend QPU Time: 0.026715569999999998
Overhead Time: 4.047571367713623

Received Task from D-Wave:
2024-02-05 03:13:26
Backend QPU Time: 0.02505037
Overhead Time: 2.965813430048828

Received Task from D-Wave:
2024-02-05 03:13:30
Backend QPU Time: 0.027796369999999997
Overhead Time: 3.8288334784802247

Received Task from D-Wave:
2024-02-05 03:13:33
Backend QPU Time: 0.02686357
Overhead Time: 3.049283271049194

Received Task from D-Wave:
2024-02-05 03:13:37
Backend QPU Time: 0.031159569999999998
Overhead Time: 3.402001450278931

Received Task from D-Wave:
2024-02-05 03:13:40
Backend QPU Time: 0.030483169999999997
Overhead Time: 3.3016107840863036

Received Task from D-Wave:
2024-02-05 03:13:43
Backend QPU Time: 0.0